In [20]:
from pykrx import stock
import pandas as pd
import numpy as np
# SK하이닉스 종목코드
ticker = "047560"
# 시작 날짜와 종료 날짜 설정
start_date = "2014-03-20"
end_date = "2024-03-20"
# 주어진 기간 동안의 일별 거래량 정보 가져오기
df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)


def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """MACD 및 MACD 신호 계산"""
    df['EMA_short'] = df['종가'].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df['종가'].ewm(span=long_window, adjust=False).mean()
    df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    return df

def calculate_rsi(df, window=14, signal_window=9):
    """RSI 및 RSI 신호 계산"""
    delta = df['종가'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # RSI 신호선 추가
    df['RSI_Signal'] = df['RSI'].ewm(span=signal_window, adjust=False).mean()
    return df

# MACD 및 MACD 신호 계산
df = calculate_macd(df)

df = calculate_rsi(df)

# '종가' 컬럼이 포함된 DataFrame을 가정합니다. 예를 들어, df라고 합시다.
# df = pd.read_csv('path_to_your_data.csv') # 데이터 파일을 로드하는 예시

# 지정된 기간에 대한 SMA 계산
periods = [5, 20, 60, 120, 240]
for period in periods:
    df[f'SMA_{period}'] = df['종가'].rolling(window=period).mean()

# 지정된 기간에 대한 EMA 계산
for period in periods:
    df[f'EMA_{period}'] = df['종가'].ewm(span=period, adjust=False).mean()

for i in range(len(periods)):
    for j in range(i + 1, len(periods)):
        smaller_period = periods[i]
        larger_period = periods[j]
        df[f'SMA_{smaller_period}_minus_SMA_{larger_period}'] = df[f'SMA_{smaller_period}'] - df[f'SMA_{larger_period}']

# EMA 간의 차이 계산
for i in range(len(periods)):
    for j in range(i + 1, len(periods)):
        smaller_period = periods[i]
        larger_period = periods[j]
        df[f'EMA_{smaller_period}_minus_EMA_{larger_period}'] = df[f'EMA_{smaller_period}'] - df[f'EMA_{larger_period}']

for column in ['시가', '고가', '저가','거래량','MACD','MACD_Signal','RSI','RSI_Signal']:
    df[f'{column}_등락률'] = df[column].pct_change() * 100

# SMA 및 EMA의 전 거래일 대비 등락률 계산 및 DataFrame에 추가
periods = [5, 20, 60, 120, 240]

# SMA 등락률 계산 및 추가
for period in periods:
    df[f'SMA_{period}_등락률'] = df[f'SMA_{period}'].pct_change() * 100

# EMA 등락률 계산 및 추가
for period in periods:
    df[f'EMA_{period}_등락률'] = df[f'EMA_{period}'].pct_change() * 100

# 'MACD'가 0 이상일 때 1, 아니면 0을 할당
df['MACD_Binary'] = (df['MACD'] >= 0).astype(int)

# 'MACD - MACD_Signal'이 0 이상일 때 1, 아니면 0을 할당
df['MACD_minus_Signal_Binary'] = ((df['MACD'] - df['MACD_Signal']) >= 0).astype(int)

# 'RSI - RSI_Signal'이 0 이상일 때 1, 아니면 0을 할당
df['RSI_minus_Signal_Binary'] = ((df['RSI'] - df['RSI_Signal']) >= 0).astype(int)

# 시가 대비 종가 등락률 컬럼 추가
df['시가_대비_종가_등락률'] = ((df['종가'] - df['시가']) / df['시가']) * 100
df['시가_대비_저가_등락률'] = ((df['저가'] - df['시가']) / df['시가']) * 100
df['시가_대비_고가_등락률'] = ((df['고가'] - df['시가']) / df['시가']) * 100
df['저가_대비_종가_등락률'] = ((df['종가'] - df['저가']) / df['저가']) * 100
df['저가_대비_고가_등락률'] = ((df['고가'] - df['저가']) / df['저가']) * 100
df['고가_대비_종가_등락률'] = ((df['종가'] - df['고가']) / df['고가']) * 100
# 종가 - SMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
for period in [5, 20, 60, 120, 240]:
    df[f'종가_minus_SMA_{period}'] = df['종가'] - df[f'SMA_{period}']

# 종가 - EMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
for period in [5, 20, 60, 120, 240]:
    df[f'종가_minus_EMA_{period}'] = df['종가'] - df[f'EMA_{period}']


csv_file_path = "C:/apps/h1/이스트소프트_data.csv"  # 저장할 파일 경로 및 이름 설정
df.to_csv(csv_file_path, encoding='utf-8-sig')

print(f"Data saved to {csv_file_path}")

Data saved to C:/apps/h1/이스트소프트_data.csv


In [23]:
df.head()

,시가,고가,저가,종가,거래량,등락률,EMA_short,EMA_long,MACD,MACD_Signal,...,종가_minus_SMA_5,종가_minus_SMA_20,종가_minus_SMA_60,종가_minus_SMA_120,종가_minus_SMA_240,종가_minus_EMA_5,종가_minus_EMA_20,종가_minus_EMA_60,종가_minus_EMA_120,종가_minus_EMA_240
날짜,,,,,,,,,,,,,,,,,,,,,
2014-03-20,9400,9550,9275,9275,13324,-2.368421,9275.000000,9275.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
2014-03-21,9275,9500,9275,9450,17620,1.886792,9301.923077,9287.962963,13.960114,2.792023,...,NaN,NaN,NaN,NaN,NaN,116.666667,158.333333,169.262295,172.107438,173.547718
2014-03-24,9425,9925,9425,9550,49193,1.058201,9340.088757,9307.373114,32.715644,8.776747,...,NaN,NaN,NaN,NaN,NaN,144.444444,233.730159,260.434023,267.609794,271.277612
2014-03-25,9600,9600,9125,9350,27521,-2.094241,9341.613564,9310.530661,31.082903,13.237978,...,NaN,NaN,NaN,NaN,NaN,-37.037037,30.517763,58.452580,66.492277,70.686097
2014-03-26,9500,9550,9275,9300,10164,-0.534759,9335.211477,9309.750612,25.460865,15.682556,...,-85.0,NaN,NaN,NaN,NaN,-58.024691,-17.626786,8.175446,16.219678,20.514428


In [26]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor



# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = df['등락률'].shift(-1)
df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else -1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

df1= df.iloc[:-500, 1:]
X = df1.drop(['next_day_return','target'], axis=1)
y = df1['target']


#X = df2.drop(['날짜', 'next_day_return','target'], axis=1)
#y = df2['target']

# Step 5: Correlation for Forward Selection
correlations = X.corrwith(y).abs().sort_values(ascending=False)

# Step 6: T-test for mean differences between label groups
t_tests = {column: ttest_ind(X[column][y == 1], X[column][y == -1], nan_policy='omit') for column in X.columns}
t_tests_sorted = sorted(t_tests.items(), key=lambda x: x[1].pvalue)

# Step 7: Lasso and Ridge for embedded methods
lasso = LassoCV().fit(X, y)
ridge = RidgeCV().fit(X, y)

lasso_importance = np.abs(lasso.coef_)
ridge_importance = np.abs(ridge.coef_)

features_lasso_sorted = sorted(zip(X.columns, lasso_importance), key=lambda x: x[1], reverse=True)
features_ridge_sorted = sorted(zip(X.columns, ridge_importance), key=lambda x: x[1], reverse=True)

# Step 8: Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True) 


c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.680460337851855, tolerance: 0.1357686817188638
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.983509380951773, tolerance: 0.1357686817188638
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.5839458047787502, tolerance: 0.1339332847778587
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_mode

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid